This notebook implements strategy number 2.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.grid import Grid
import random
from ipywidgets import IntProgress
from IPython.display import display
%matplotlib inline

As input data we use consumption/production of specific houses, that were found to have reasonable consumption/production quantities. Also we set the timeframe to a short one of two weeks, such that the genetic algorithm runs faster than for a whole year.

In [2]:
# Consumption data
df_cons = pd.read_csv('data/consumption_ausgrid.csv')
df_cons['Unnamed: 0'] = pd.to_datetime(df_cons['Unnamed: 0'])
df_cons = df_cons.set_index('Unnamed: 0')
df_cons_red = df_cons.iloc[:,0]
# Production data
df_prod = pd.read_csv('data/generation_ausgrid.csv')
df_prod['Unnamed: 0'] = pd.to_datetime(df_prod['Unnamed: 0'])
df_prod = df_prod.set_index('Unnamed: 0')

# Choose specific houses and time frame
used_houses = [114,204,47,160,82,168]
week_start = 37
week_end = 39
df_cons = df_cons.iloc[7*48*week_start:7*48*week_end,used_houses]
df_prod = df_prod.iloc[7*48*week_start:7*48*week_end,used_houses]

data_cons = df_cons.values
data_prod = df_prod.values

Now we can build the grid model. Since we only simulate 2 weeks, we need to set the costs accordingly. Therfore we define the timeframe after which the storages should pay off (i.e. 10 years) and calculate its investition cost for two weeks. 

In [3]:
g = Grid(num_houses=df_cons.shape[1],
         num_storages=1,
         max_capacity=0,
         num_pvtypes=df_prod.shape[1],
         pv_peakpower=0)
# Add each pv type to each house
for i in range(df_cons.shape[1]):
    g.change_pvtype(num_house=i,pv_type=i)

# Set the costs for 2 weeks
amortization_timeframe = 10*365 # in days
storage_cost_per_kWh = 500
storage_cost_per_kWh_per_day = storage_cost_per_kWh / amortization_timeframe
g.set_costs(cost_storage_kwh=14*storage_cost_per_kWh_per_day)

# Simulate the grid
res_dict = g.simulate(data_cons=data_cons,data_prod=data_prod)
print(res_dict)

{'import_grid_kwh': 471.50200000000046, 'export_grid_kwh': 624.861, 'pv_production_kwh': 880.918, 'setup_cost_storage': 0.0, 'setup_cost_pv': 0.0, 'cost_import_grid': 117.87550000000012, 'reward_export_grid': 62.4861}


As we can see there is something going on with pv production and export. We still need an objective function that returns the cost for running this kind of grid for the specific timeframe. In this case the objective function is really easy, since we already defined the cost for setting up the storages in this two week interval.

In [4]:
# TODO Add a function to grid.py that empties the storages at the end of the simulation.
def objective_function(res_dict):
    return res_dict['cost_import_grid'] + res_dict['setup_cost_storage'] - res_dict['reward_export_grid']

# Calculate the cost of the above simulation
print('Cost without storage: ' + str(objective_function(res_dict)))

Cost without storage: 55.389400000000116


Now we can setup the storage system and calculate the expenses again.

In [5]:
g.change_storages(num_storages=1,max_capacity=14)
g.print()
res_dict = g.simulate(data_cons=data_cons,data_prod=data_prod)
print('Cost with storages (initial): ' + str(objective_function(res_dict)))

pv_types: [0 1 2 3 4 5]
pv_peakpowers: [0 0 0 0 0 0]
storag_conns: [0 0 0 0 0 0]
storage_capacities: [14]
Cost with storages (initial): 47.56156506849317


In [7]:
int(1.5+0.1)

1